In [1]:
import json
from urllib.parse import urlencode
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import requests
from lxml import html

import pickle


In [2]:
def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}

    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(pd.DataFrame(d, columns=h))

Start by exploring combine data

In [5]:
params = {'LeagueID':'00',
          'Season':'2017-18',
          'IsOnlyCurrentSeason':'0'
         }
data = get_nba_data(endpt="commonallplayers",params=params)
data['PLAYER_NAME']=pd.Series(data['DISPLAY_FIRST_LAST'],dtype=str)+"("+pd.Series(data.FROM_YEAR,dtype=str)+")"
data = data.set_index('PLAYER_NAME')
data['PLAYER_ID']=data['PERSON_ID']
Player_ID_Dict = pd.Series(data['PLAYER_ID'],dtype=int)
Player_ID_Dict
Player_ID_Dict = Player_ID_Dict.loc[~Player_ID_Dict.index.duplicated(keep='first')]

pickle.dump(Player_ID_Dict, open('NBAPlayerDict.pkl', 'wb'))

^C


IndexError: list index out of range

In [ ]:
year = "2012-13"
params = {"LeagueID":"00",
          "SeasonYear":year}
endpt = "draftcombinedrillresults"
df1 = get_nba_data(endpt=endpt,params=params)

In [ ]:
endpt = "draftcombineplayeranthro"
df2 = get_nba_data(endpt=endpt,params=params)

In [ ]:
year = "2013-14"
params = {"LeagueID":"00",
          "SeasonYear":year}
endpt = "draftcombinenonstationaryshooting"
df3 = get_nba_data(endpt=endpt,params=params)

In [ ]:
df = pd.merge(df1,df2,on="PLAYER_ID")

Not looking like we'll have the shooting data very constitently. Lets merge two we have consitently and compile it over several years

In [6]:
dfCombineFull = pd.DataFrame(columns=df.columns)
for i in range(10,13):
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({"SeasonYear":year})
    df1 = get_nba_data(endpt="draftcombinedrillresults",params=params)
    df2 = get_nba_data(endpt="draftcombineplayeranthro",params=params)
    dfMerged = pd.merge(df1,df2,on="PLAYER_ID")

    if(i>13):
        dfMerged = pd.merge(df1,df2,on="TEMP_PLAYER_ID")
        dfMerged['PLAYER_NAME'] = dfMerged['PLAYER_NAME_x'] + "(" + year[0:4] + ")"
        dfMerged['PLAYER_ID']=dfMerged['PLAYER_NAME'].map(Player_ID_Dict)
        dfMerged['PLAYER_ID']=pd.Series(dfMerged['PLAYER_ID'],dtype=str).str.split(".").str.get(0)

    dfCombineFull = pd.concat([dfCombineFull,dfMerged])

NameError: name 'df' is not defined

In [7]:
#dfCombineFull = dfCombineFull.set_index('PLAYER_ID')
dfCombineFull[dfCombineFull.LAST_NAME_x=="Davis"]

NameError: name 'dfCombineFull' is not defined

In [127]:
endpt = 'playerdashptshotdefend'
params = {'DateFrom':'','DateTo':'','GameSegment':'',
          'LastNGames':'0','LeagueID':'00','Location':'',
          'Month':'0','OpponentTeamID':'0','Outcome':'',
          'Period':'0','PlayerID':'0','Season':year,
          'SeasonSemgent':'','SeasonType':'Regular Season',
          'TeamID':'0','VsConference':'','VsDivision':'',
          'PerMode':'Totals','SeasonSegment':''
         }
dfDefense = get_nba_data(endpt=endpt,params=params)
dfDefense[dfDefense.DEFENSE_CATEGORY=="Less Than 6 Ft"]


IndexError: list index out of range

In [11]:
dfDefense = dfDefense[dfDefense.DEFENSE_CATEGORY=="Less Than 6 Ft"]
dfDefense = dfDefense.set_index("CLOSE_DEF_PERSON_ID")

NameError: name 'dfDefense' is not defined

In [12]:
dfDefense_PCT_Change = pd.DataFrame(dfDefense.PCT_PLUSMINUS)
dfDefense_PCT_Change

NameError: name 'dfDefense' is not defined

In [13]:
dfCombineFull = pd.merge(dfCombineFull,dfDefense_PCT_Change,left_index=True,right_index=True)

NameError: name 'dfDefense_PCT_Change' is not defined

In [14]:
dfCombineFull.groupby('WINGSPAN')['PCT_PLUSMINUS'].mean()

KeyError: 'Column not found: PCT_PLUSMINUS'

In [15]:
plt.plot(dfCombineFull.groupby('WINGSPAN')['PCT_PLUSMINUS'].mean())

KeyError: 'Column not found: PCT_PLUSMINUS'

In [16]:
bins = list(range(70,98,2))
dfCombineFull['Wingspan_binned']=pd.cut(dfCombineFull['WINGSPAN'],bins)

In [17]:
dfCombineFull['POSITION_SIMPLE']=np.NaN
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="C"]="Big"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="C-PF"]="Big"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="PF-C"]="Big"

dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="PF"]="Forward"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="SF-PF"]="Forward"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="PF-SF"]="Forward"

dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="SF"]="Wing"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="SF-SG"]="Wing"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="SG-SF"]="Wing"

dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="SG"]="Guard"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="PG-SG"]="Guard"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="SG-PG"]="Guard"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="PG"]="Guard"




dfCombineFull.groupby(['POSITION_SIMPLE','Wingspan_binned'])['PCT_PLUSMINUS'].mean()


/home/dhoskins/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/dhoskins/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dhoskins/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/dh

KeyError: 'Column not found: PCT_PLUSMINUS'

In [ ]:
plt.scatter(dfCombineFull['THREE_QUARTER_SPRINT'],dfCombineFull['PCT_PLUSMINUS'])

In [ ]:
dfCombineFull.columns

Tomorrow we should work on a simple linear regression from this.
We should also start adding in college status
Need to do data from multiple years/figure out what we want to do regarding that
    averages over full career
    best year
    three years out?
    three year average?

In [8]:
year = 16
year="20"+format(i,'02d')+"-"+format(i+1,'02d')
params.update({"SeasonYear":year})
df1 = get_nba_data(endpt="draftcombinedrillresults",params=params)
df2 = get_nba_data(endpt="draftcombineplayeranthro",params=params)
if(i>13):
    dfMerged = pd.merge(df1,df2,on="TEMP_PLAYER_ID")
    
dfMerged['PLAYER_NAME'] = dfMerged['PLAYER_NAME_x'] + "(" + year[0:4] + ")"
dfMerged['PLAYER_ID']=dfMerged['PLAYER_NAME'].map(Player_ID_Dict)
dfMerged['PLAYER_ID']=pd.Series(dfMerged['PLAYER_ID'],dtype=str).str.split(".").str.get(0)


dfCombineFull = pd.concat([dfCombineFull,dfMerged])
dfMerged

NameError: name 'i' is not defined

In [9]:
Player_ID_Dict = Player_ID_Dict.loc[~Player_ID_Dict.index.duplicated(keep='first')]

NameError: name 'Player_ID_Dict' is not defined

In [10]:
Player_ID_Dict['Anthony Davis(20)']

NameError: name 'Player_ID_Dict' is not defined

Going to start pulling college stats from bball ref but might get my IP banned which is less than ideal.

In [11]:
nameList = []
for player in dfCombineFull.index:
    nameStr = 'https://www.sports-reference.com/cbb/players/'
    
    nameStr += str(dfCombineFull.FIRST_NAME_x[player]).lower() +"-" + \
                str(dfCombineFull.LAST_NAME_x[player]).lower() + "-1.html"
    
    nameList.append(nameStr)

NameError: name 'dfCombineFull' is not defined

In [12]:
testCase = 'https://www.sports-reference.com/cbb/players/deandre-ayton-1.html'

In [13]:
data_url = 'https://www.sports-reference.com/cbb/players/malcolm-brogdon-1.html'
user_agent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""

headers = {'User-Agent':user_agent}
page = requests.get(data_url,headers=headers)
tree = html.fromstring(page.content)

In [14]:

x = tree.xpath('//div[@id="all_players_per_poss"]')
n = len(str(x[0].getchildren()[-1]))

tree2 = html.fromstring(str(x[0].getchildren()[-1])[6:(n-3)])
tree2.xpath('//table/tfoot/tr/td/text()')

['Virginia',
 '136',
 '109',
 '4157',
 '9.6',
 '22.3',
 '.430',
 '6.6',
 '14.2',
 '.466',
 '2.9',
 '8.1',
 '.365',
 '6.7',
 '7.7',
 '.876',
 '9.0',
 '5.3',
 '1.9',
 '0.5',
 '3.3',
 '4.0',
 '28.8',
 '114.9',
 '94.0']

In [15]:
x = tree.xpath('//div[@id="all_leaderboard"]')
n = len(str(x[0].getchildren()[-1]))

tree2 = html.fromstring(str(x[0].getchildren()[-1])[6:(n-3)])
tree2.xpath('//table/tr/td/a/text')

[]

In [16]:
x = soup.find_all('td')

NameError: name 'soup' is not defined

In [17]:
//*[@id="all_leaderboard"]

SyntaxError: invalid syntax (<ipython-input-17-fc43eab94dfd>, line 1)

In [18]:
nameList

[]

In [19]:
#dfCombineFull = pd.DataFrame(columns=df.columns)
i = 17
year="20"+format(i,'02d')+"-"+format(i+1,'02d')
params.update({"SeasonYear":year})
df1 = get_nba_data(endpt="draftcombinedrillresults",params=params)
df2 = get_nba_data(endpt="draftcombineplayeranthro",params=params)


dfMerged = pd.merge(df1,df2)

if(i>13):
    dfMerged['PLAYER_NAME'] = dfMerged['PLAYER_NAME'] + "(" + year[0:4] + ")"
    dfMerged['PLAYER_ID']=dfMerged['PLAYER_NAME'].map(Player_ID_Dict)
    dfMerged['PLAYER_ID']=pd.Series(dfMerged['PLAYER_ID'],dtype=str).str.split(".").str.get(0)

    
dfMerged = dfMerged[dfMerged['PLAYER_ID']!='nan']

dfMerged = dfMerged[dfMerged['MAX_VERTICAL_LEAP']!=np.nan]\
                    .set_index('PLAYER_ID')\
                    .drop(['FIRST_NAME','LAST_NAME'],axis=1)
#.set_index('PLAYER_ID')
#dfCombineFull = pd.concat([dfCombineFull,dfMerged])
dfMerged

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
nameList = []
yearList = []
for name in dfMerged.PLAYER_NAME:
    nameStr = 'https://www.sports-reference.com/cbb/players/'
    first = re.sub('[^a-zA-Z]+', '',name.split()[0].lower())
    last = "".join(name.split()[1:])[0:-6].lower().strip(".")
    year = name.split()[1][-5:-1]
    yearList.append(year)
    nameStr += first + "-" + last + "-"
    nameList.append(nameStr)
    


NameError: name 'dfMerged' is not defined

In [21]:
nameList

[]

In [22]:
rowDf = pd.DataFrame(columns=row.columns)
for indx,name in enumerate(nameList):
    stopCondition = False
    year = yearList[indx]
    i = 1

    while not stopCondition:
        data_url = name + str(i) + ".html"
        print(data_url)
        player_id = dfMerged.index[indx]
        player_name = dfMerged.PLAYER_NAME[indx]
        user_agent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""

        headers = {'User-Agent':user_agent}
        page = requests.get(data_url,headers=headers)
        tree = html.fromstring(page.content)

        if '404' not in tree.xpath('//title')[0].text:
            finalYear = tree.xpath('//table/tbody/tr')[-1].items()[0][-1][-4:]

            if (finalYear != year):
                i+=1
                continue
            else:
                stopCondition = True

            treeStr = str(tree.xpath('//div[@id="all_players_per_poss"]')[0]\
                            .getchildren()[-1])[8:-4]

            tree2 = html.fromstring(treeStr)
            valuesHTML = tree2.xpath('//table/tfoot/tr')[-1].getchildren()[1:]
            conf = tree2.xpath('//table/tbody/tr/td')[1].getchildren()[0].text
            colNamesHTML = tree2.xpath('//table/thead/tr/th')
            colNames = ["PLAYER_ID","PLAYER_NAME"]+["CONF"] + [col.text for col in colNamesHTML][1:]
            values = [player_id,player_name] + [conf] + [val.text for val in valuesHTML]

            row = pd.DataFrame(pd.Series(values,index=colNames)\
                    .dropna()).transpose().set_index("PLAYER_ID")

        else:
            stopCondition=True
            row = None
    rowDf = pd.concat([rowDf,row])
    print(indx)
    
#Fixing juniors is going to require a functional approach to this. 

NameError: name 'row' is not defined

In [23]:
len(dfMerged)

NameError: name 'dfMerged' is not defined

Need to make that into a function soon. Fuck it why not now.

In [6]:
def SportsRefCBBRowMaker(player_url,table_name,player_name,correct_year):
    player_id = Player_ID_Dict[player_name]
    
    page = requests.get(player_url,headers=headers)
    tree = html.fromstring(page.content)
    
    if '404' in tree.xpath('//title')[0].text:
        return('404')
    
    else:
        year = tree.xpath('//table/tbody/tr')[-1].items()[0][-1][-4:]
        if(year!=correct_year):
            return(year)
        path = '//div[@id="'+table_name+'"]'
        treeStr = str(tree.xpath(path)[0].getchildren()[-1])[8:-4]
        #This is tricky and comments might be nice here. 

        tree2 = html.fromstring(treeStr)
        
        valuesHTML = tree2.xpath('//table/tfoot/tr')[-1].getchildren()[1:]
        ## This gives career in particular 
        conf = tree2.xpath('//table/tbody/tr/td')[1].getchildren()[0].text
        colNamesHTML = tree2.xpath('//table/thead/tr/th')
        colNames = ["PLAYER_ID","PLAYER_NAME"]+["CONF"] + [col.text for col in colNamesHTML][1:]
        values = [player_id,player_name] + [conf] + [val.text for val in valuesHTML]

        row = pd.DataFrame(pd.Series(values,index=colNames)\
                .dropna()).transpose().set_index("PLAYER_ID")
 #       print(player_url)
        return(row)
    

In [7]:
def sportsRefCBBTableMaker(player_df, table_name):
    base_url = 'https://www.sports-reference.com/cbb/players/'
    columns_df_url = "https://www.sports-reference.com/cbb/players/malcolm-brogdon-1.html"
    columns_df = SportsRefCBBRowMaker(columns_df_url,table_name
                                      ,"Malcolm Brogdon(2016)",'2016')
    CBB_df = pd.DataFrame(columns=columns_df.columns)
    for index,row in dfMerged.iterrows():
        player_name = row['PLAYER_NAME']
        first = re.sub('[^a-zA-Z]+', '',player_name.split()[0].lower())
        last = "".join(player_name.split()[1:])[0:-6].lower().strip(".")
        year = player_name.split()[1][-5:-1]
        name_url = base_url + first + "-" + last + "-"

        i = 1
        stopping_condition=False
        while(stopping_condition==False):
            data_url = name_url + str(i) + ".html"
#            print(data_url)
            row = SportsRefCBBRowMaker(data_url,table_name,player_name,year)
            if isinstance(row, pd.DataFrame):
                stopping_condition=True
                continue
            if(row=='404'):
                stopping_condition=True
                row = None
                if(last[-2:]=="jr"):
                    last = last[:-2]
                    name_url = base_url + first + "-" + last + "-"
                    i=1
                    stopping_condition = False 
            i+=1
        CBB_df = pd.concat([CBB_df,row])
    return(CBB_df)

In [8]:
sportsRefCBBTableMaker(dfMerged, "all_players_per_poss")

NameError: name 'dfMerged' is not defined

In [9]:
dfCombineFull = pd.DataFrame(columns=df.columns)
for i in range(10,13):
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({"SeasonYear":year})
    df1 = get_nba_data(endpt="draftcombinedrillresults",params=params)
    df2 = get_nba_data(endpt="draftcombineplayeranthro",params=params)
    dfMerged = pd.merge(df1,df2,on="PLAYER_ID")

    if(i>13):
        dfMerged = pd.merge(df1,df2,on="TEMP_PLAYER_ID")
        dfMerged['PLAYER_NAME'] = dfMerged['PLAYER_NAME_x'] + "(" + year[0:4] + ")"
        dfMerged['PLAYER_ID']=dfMerged['PLAYER_NAME'].map(Player_ID_Dict)
        dfMerged['PLAYER_ID']=pd.Series(dfMerged['PLAYER_ID'],dtype=str).str.split(".").str.get(0)

    dfCombineFull = pd.concat([dfCombineFull,dfMerged])

NameError: name 'df' is not defined

In [10]:
full_combine_df = pd.DataFrame(columns = combine_columns)
combine_columns = ['PLAYER_NAME','DRAFT_YEAR','POSITION','STANDING_VERTICAL_LEAP',
                   'MAX_VERTICAL_LEAP','LANE_AGILITY_TIME',
                   'MODIFIED_LANE_AGILITY_TIME','THREE_QUARTER_SPRINT',
                   'BENCH_PRESS','HEIGHT_WO_SHOES','WEIGHT','WINGSPAN',
                   'STANDING_REACH','BODY_FAT_PCT','HAND_LENGTH','HAND_WIDTH'                  
                  ]
for i in range(0,19):
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({"SeasonYear":year})
    df1 = get_nba_data(endpt="draftcombinedrillresults",params=params)
    df2 = get_nba_data(endpt="draftcombineplayeranthro",params=params)
    dfMerged = pd.merge(df1,df2)
    dfMerged['PLAYER_ID']=pd.Series(dfMerged.PLAYER_ID,dtype=str)
    if(i>13):
        dfMerged['PLAYER_NAME'] = dfMerged['PLAYER_NAME']
        dfMerged['PLAYER_ID']=(dfMerged['PLAYER_NAME']+"("+year[0:4]+")").map(Player_ID_Dict)
        dfMerged['PLAYER_ID']=pd.Series(dfMerged['PLAYER_ID'],dtype=str).str.split(".").str.get(0)

    dfMerged['DRAFT_YEAR'] = int(year[0:4])    
    dfMerged = dfMerged[dfMerged['PLAYER_ID']!='nan']

    dfMerged = dfMerged[dfMerged['MAX_VERTICAL_LEAP']!=np.nan]\
                        .set_index('PLAYER_ID')[combine_columns]
        
    full_combine_df = pd.concat([full_combine_df,dfMerged])
full_combine_df.fillna(value=np.nan, inplace=True)
full_combine_df

NameError: name 'combine_columns' is not defined

In [ ]:
full_combine_df.groupby('DRAFT_YEAR').count()

In [ ]:
full_combine_df

Lets simply build a full NBA Season database over last however many years.
Probably indexed on player ID, season, with years experience in here somewhere. 



In [ ]:
x = get_nba_data(endpt="drafthistory",params={'LeagueID':'00'})
x[x.SEASON=='2016'].head()

Using schools and draft data can save a lot of requests perhaps. 

In [ ]:
x = get_nba_data(endpt="leaguedashlineups",params={'LeagueID':'00'},
                return_url=True)
x

First approach to a comprehensive nba database will follow stats used in the clustering analysis I am to follow.

We're going to follow of this medium post which was referenced on HW3.
https://fastbreakdata.com/classifying-the-modern-nba-player-with-machine-learning-539da03bb824

My man used selium to scrape from bbal reference which is probably smarter than what I did. I turn to stats.nba as the API is pretty usable.

He uses per 100 possesion, advanced and shooting data. I shall use the same but add in some defensive matchup data. 

He used career which I am going to not do. I'll use last 3 years or something. Perhaps limited by the player tracking. I also will add in some team data as what kind of offense you run and things likely has a big impact on what position you'll be playing? Maybe I shouldn't do this actually. Will consider.

Lets start building our giant beautiful database and then define a bunch of variables that can be used to selectively choose columns.

We'll use regular seson data but a playoff data analysis might be more fun later on for somehing else idk.


Since we're going to be doing a somewhat decent amount of figuring out parameters through looking at what exists on the nba site I'm going to define a little function here.

In [11]:
url = 'http://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2017-18&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
    
def get_params(url):
    endpt = url.split('stats/')[1].split('?')[0]
    params = {}
    paramstring = url.split('stats/')[1].split('?')[1]

    for substring in paramstring.split("&"):
        variable = substring.split('=')[0]
        value = substring.split('=')[1]
        params.update({variable:value})
    return([endpt,params])
endpt,params = get_params(url)
params.update({'SeasonType':'Regular Season'})
print(params)
advanced_2017_18 = get_nba_data(endpt=endpt,params=params)

{'College': '', 'Conference': '', 'Country': '', 'DateFrom': '', 'DateTo': '', 'Division': '', 'DraftPick': '', 'DraftYear': '', 'GameScope': '', 'GameSegment': '', 'Height': '', 'LastNGames': '0', 'LeagueID': '00', 'Location': '', 'MeasureType': 'Advanced', 'Month': '0', 'OpponentTeamID': '0', 'Outcome': '', 'PORound': '0', 'PaceAdjust': 'N', 'PerMode': 'PerGame', 'Period': '0', 'PlayerExperience': '', 'PlayerPosition': '', 'PlusMinus': 'N', 'Rank': 'N', 'Season': '2017-18', 'SeasonSegment': '', 'SeasonType': 'Regular Season', 'ShotClockRange': '', 'StarterBench': '', 'TeamID': '0', 'VsConference': '', 'VsDivision': '', 'Weight': ''}


In [12]:
advanced_2017_18['SEASON'] = 2018
advanced_2017_18['SEASON_TYPE']='Regular Season'

keep = ['PLAYER_ID','SEASON','SEASON_TYPE','PLAYER_NAME','TEAM_ID','TEAM_ABBREVIATION','AGE','GP','MIN','OFF_RATING','DEF_RATING',
    'AST_PCT','AST_TO','AST_RATIO','OREB_PCT','DREB_PCT','REB_PCT','TM_TOV_PCT','EFG_PCT',
    'TS_PCT','USG_PCT','PACE']
advanced_2017_18 = advanced_2017_18[keep].set_index(['PLAYER_ID','SEASON','SEASON_TYPE'])


# .loc[:,~advanced_2017_18.columns.str.contains('RANK')]\
#                                     .drop(['CFID','CFPARAMS','W_PCT','W','L','PIE',
#                                            'FGM','FGA','FGM_PG','FGA_PG'                                          
#                                           ],axis=1)\


Back to 2013-14 begninning of  player tracking data. 
Playtype might be important and that only goes so far as 2015. 

This isn't especially important because we just need some sort of clustering.


In [13]:
advanced_full = pd.DataFrame(columns=advanced_2017_18.columns)

for i in range(13,18):
    params.update({'SeasonType':'Regular Season'})
    
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({'Season':year})
    reg_season = get_nba_data(endpt=endpt,params=params)
    reg_season['SEASON_TYPE'] = 'Regular Season'
    params.update({'SeasonType':'Playoffs'})
    
    playoff = get_nba_data(endpt=endpt,params=params)
    playoff['SEASON_TYPE'] = 'Playoffs'
    merged = pd.concat([reg_season,playoff])
    
    merged['SEASON']= 2001+i
    
    merged = merged[keep]

    advanced_full = pd.concat([advanced_full,merged])

newcols = advanced_full.columns[advanced_full.columns=='PLAYER_NAME'].append(
advanced_full.columns[advanced_full.columns!='PLAYER_NAME'])
advanced_full['SEASON']=pd.Series(advanced_full.SEASON,dtype='int64')

advanced_full['PLAYER_ID']=pd.Series(advanced_full.PLAYER_ID,dtype='int64')
advanced_full['PLAYER_ID']=pd.Series(advanced_full.PLAYER_ID,dtype='str')

advanced_full = advanced_full[newcols]\
                        .set_index(['PLAYER_ID','SEASON','SEASON_TYPE'])

In [14]:
advanced_full.sort_index().head()

PLAYER_NAME   AGE  AST_PCT  AST_RATIO  \
PLAYER_ID SEASON SEASON_TYPE                                              
101106    2014   Regular Season  Andrew Bogut  29.0    0.087       18.3   
          2015   Playoffs        Andrew Bogut  30.0    0.121       25.2   
                 Regular Season  Andrew Bogut  30.0    0.149       27.3   
          2016   Playoffs        Andrew Bogut  31.0    0.127       22.7   
                 Regular Season  Andrew Bogut  31.0    0.145       29.7   

                                 AST_TO  DEF_RATING  DREB_PCT  EFG_PCT  GP  \
PLAYER_ID SEASON SEASON_TYPE                                                 
101106    2014   Regular Season    1.15        98.8     0.293    0.627  67   
          2015   Playoffs          1.38        98.9     0.275    0.560  19   
                 Regular Season    1.70        95.2     0.262    0.563  67   
          2016   Playoffs          1.58        95.4     0.203    0.623  22   
                 Regular Season    1.95        97.2     0.252    0.629  70   

                                  MIN  OFF_RATING  OREB_PCT    PACE  REB_PCT  \
PLAYER_ID SEASON SEASON_TYPE                                                   
101106    2014   Regular Season  26.4       107.8     0.117   98.50    0.208   
          2015   Playoffs        23.1       101.8     0.107   93.93    0.193   
                 Regular Season  23.6       111.7     0.101   99.56    0.185   
          2016   Playoffs        16.6       105.9     0.134   98.08    0.170   
                 Regular Season  20.7       111.4     0.094  101.07    0.178   

                                TEAM_ABBREVIATION     TEAM_ID  TM_TOV_PCT  \
PLAYER_ID SEASON SEASON_TYPE                                                
101106    2014   Regular Season               GSW  1610612744        15.8   
          2015   Playoffs                     GSW  1610612744        18.2   
                 Regular Season               GSW  1610612744        16.1   
          2016   Playoffs                     GSW  1610612744        14.4   
                 Regular Season               GSW  1610612744        15.2   

                                 TS_PCT  USG_PCT  
PLAYER_ID SEASON SEASON_TYPE                      
101106    2014   Regular Season   0.610    0.124  
          2015   Playoffs         0.551    0.113  
                 Regular Season   0.565    0.133  
          2016   Playoffs         0.607    0.125  
                 Regular Season   0.623    0.116

In [54]:
endpt,params = get_params('http://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Per100Possessions&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2017-18&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight=')
params.update({'SeasonType':'Regular Season'})
per100poss = get_nba_data(endpt,params)
per100poss['SEASON'] = 2018
per100poss['SEASON_TYPE']='Regular Season'
per100poss = per100poss.loc[:,~per100poss.columns.str.contains('RANK')]\
                                    .drop(['CFID','CFPARAMS','NBA_FANTASY_PTS',
                                           'DD2','TD3','W','L','W_PCT','MIN',
                                           'PLAYER_NAME','TEAM_ID','TEAM_ABBREVIATION',
                                           'AGE','GP'
                                          ]
                                          ,axis=1)\
                                    .set_index(['PLAYER_ID','SEASON','SEASON_TYPE'])
per100poss.columns

Index(['FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA',
       'PF', 'PFD', 'PTS', 'PLUS_MINUS'],
      dtype='object')

In [58]:
per_100p_full= pd.DataFrame(columns=per100poss.columns)

for i in range(13,18):
    params.update({'SeasonType':'Regular Season'})
    
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({'Season':year})
    reg_season = get_nba_data(endpt=endpt,params=params)
    reg_season['SEASON_TYPE'] = 'Regular Season'
    params.update({'SeasonType':'Playoffs'})
    
    playoff = get_nba_data(endpt=endpt,params=params)
    playoff['SEASON_TYPE'] = 'Playoffs'
    merged = pd.concat([reg_season,playoff])
    merged = merged.loc[:,~merged.columns.str.contains('RANK')]\
                                    .drop(['CFID','CFPARAMS','NBA_FANTASY_PTS',
                                           'DD2','TD3','W','L','W_PCT','MIN',
                                           'PLAYER_NAME','TEAM_ID','TEAM_ABBREVIATION',
                                           'AGE','GP'
                                          ],
                                          axis=1)
    merged['SEASON']= 2001+i

    per_100p_full = pd.concat([per_100p_full,merged])

newcols = per_100p_full.columns[per_100p_full.columns=='PLAYER_NAME'].append(
per_100p_full.columns[per_100p_full.columns!='PLAYER_NAME'])
per_100p_full['SEASON']=pd.Series(per_100p_full.SEASON,dtype='int64')

per_100p_full['PLAYER_ID']=pd.Series(per_100p_full.PLAYER_ID,dtype='int64')
per_100p_full['PLAYER_ID']=pd.Series(per_100p_full.PLAYER_ID,dtype='str')

per_100p_full = per_100p_full[newcols]\
                        .set_index(['PLAYER_ID','SEASON','SEASON_TYPE'])
per_100p_full.head()

,,,AST,BLK,BLKA,DREB,FG3A,FG3M,FG3_PCT,FGA,FGM,FG_PCT,...,FTM,FT_PCT,OREB,PF,PFD,PLUS_MINUS,PTS,REB,STL,TOV
PLAYER_ID,SEASON,SEASON_TYPE,,,,,,,,,,,,,,,,,,,,,
201985,2014,Regular Season,6.4,0.0,0.0,4.4,10.9,3.0,0.273,22.7,9.4,0.413,...,0.0,0.000,0.5,2.5,1.5,0.5,21.7,4.9,0.5,3.5
201166,2014,Regular Season,7.2,0.4,1.1,3.0,7.7,3.0,0.387,18.0,7.2,0.401,...,2.6,0.874,1.3,4.5,3.3,-2.9,19.9,4.3,1.6,3.6
201189,2014,Regular Season,3.1,1.1,1.4,9.7,0.1,0.0,0.000,8.6,3.8,0.443,...,1.5,0.550,5.9,9.0,2.2,-11.9,9.1,15.6,1.4,4.3
203519,2014,Regular Season,4.0,0.0,1.3,4.0,6.7,1.3,0.200,18.7,8.0,0.429,...,1.3,1.000,0.0,8.0,2.7,10.7,18.7,4.0,0.0,1.3
1733,2014,Regular Season,2.8,0.0,1.0,6.4,9.9,3.4,0.340,20.4,8.1,0.396,...,2.7,0.771,1.5,7.1,3.9,4.5,22.2,7.9,1.4,3.4


In [63]:
endpt,params = get_params('http://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Scoring&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2017-18&SeasonSegment=&SeasonType=Playoffs&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight=')
params.update({'SeasonType':'Regular Season'})
scoring = get_nba_data(endpt,params)
scoring['SEASON'] = 2018
scoring['SEASON_TYPE']='Regular Season'
scoring = scoring.loc[:,~scoring.columns.str.contains('RANK')]\
                                    .drop(['CFID','CFPARAMS','W','L','W_PCT',
                                           'PLAYER_NAME','TEAM_ID','TEAM_ABBREVIATION',
                                           'AGE','GP','MIN'
                                          ]
                                          ,axis=1)\
                                    .set_index(['PLAYER_ID','SEASON','SEASON_TYPE'])
scoring.columns

Index(['PCT_FGA_2PT', 'PCT_FGA_3PT', 'PCT_PTS_2PT', 'PCT_PTS_2PT_MR',
       'PCT_PTS_3PT', 'PCT_PTS_FB', 'PCT_PTS_FT', 'PCT_PTS_OFF_TOV',
       'PCT_PTS_PAINT', 'PCT_AST_2PM', 'PCT_UAST_2PM', 'PCT_AST_3PM',
       'PCT_UAST_3PM', 'PCT_AST_FGM', 'PCT_UAST_FGM'],
      dtype='object')

In [66]:
scoring_full= pd.DataFrame(columns=scoring.columns)

for i in range(13,18):
    params.update({'SeasonType':'Regular Season'})
    
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({'Season':year})
    reg_season = get_nba_data(endpt=endpt,params=params)
    reg_season['SEASON_TYPE'] = 'Regular Season'
    params.update({'SeasonType':'Playoffs'})
    
    playoff = get_nba_data(endpt=endpt,params=params)
    playoff['SEASON_TYPE'] = 'Playoffs'
    merged = pd.concat([reg_season,playoff])
    merged = merged.loc[:,~merged.columns.str.contains('RANK')]\
                        .drop(['CFID','CFPARAMS','W','L','W_PCT',
                                           'PLAYER_NAME','TEAM_ID','TEAM_ABBREVIATION',
                                           'AGE','GP','MIN'
                                          ]
                              ,axis=1)
    merged['SEASON']= 2001+i

    scoring_full = pd.concat([scoring_full,merged])

scoring_full['SEASON']=pd.Series(scoring_full.SEASON,dtype='int64')

scoring_full['PLAYER_ID']=pd.Series(scoring_full.PLAYER_ID,dtype='int64')
scoring_full['PLAYER_ID']=pd.Series(scoring_full.PLAYER_ID,dtype='str')

scoring_full = scoring_full.set_index(['PLAYER_ID','SEASON','SEASON_TYPE'])
scoring_full.head()

,,,PCT_AST_2PM,PCT_AST_3PM,PCT_AST_FGM,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_UAST_2PM,PCT_UAST_3PM,PCT_UAST_FGM
PLAYER_ID,SEASON,SEASON_TYPE,,,,,,,,,,,,,,,
201985,2014,Regular Season,0.231,0.833,0.421,0.522,0.478,0.591,0.409,0.409,0.114,0.000,0.114,0.182,0.769,0.167,0.579
201166,2014,Regular Season,0.139,0.698,0.369,0.573,0.427,0.425,0.099,0.447,0.109,0.129,0.138,0.326,0.861,0.302,0.631
201189,2014,Regular Season,0.667,0.000,0.667,0.984,0.016,0.831,0.092,0.000,0.031,0.169,0.108,0.738,0.333,0.000,0.333
203519,2014,Regular Season,0.200,1.000,0.333,0.643,0.357,0.714,0.286,0.214,0.429,0.071,0.357,0.429,0.800,0.000,0.667
1733,2014,Regular Season,0.646,1.000,0.793,0.517,0.483,0.427,0.053,0.453,0.098,0.120,0.164,0.373,0.354,0.000,0.207


In [19]:
year="2017-18"
endpt = 'playerdashptshotdefend'
params = {'DateFrom':'','DateTo':'','GameSegment':'',
          'LastNGames':'0','LeagueID':'00','Location':'',
          'Month':'0','OpponentTeamID':'0','Outcome':'',
          'Period':'0','PlayerID':'0','Season':year,
          'SeasonSemgent':'','SeasonType':'Regular Season',
          'TeamID':'0','VsConference':'','VsDivision':'',
          'PerMode':'Totals','SeasonSegment':''
         }
defensive = get_nba_data(endpt=endpt,params=params)
defensive['PLAYER_ID'] = defensive.CLOSE_DEF_PERSON_ID
defensive = defensive.set_index('PLAYER_ID').drop(['CLOSE_DEF_PERSON_ID'],axis=1)
defensive.groupby('DEFENSE_CATEGORY')[['PCT_PLUSMINUS','FREQ']].describe()

PCT_PLUSMINUS                                             \
                           count      mean       std    min      25%    50%   
DEFENSE_CATEGORY                                                              
2 Pointers                 529.0  0.009650  0.104892 -0.574 -0.02400  0.004   
3 Pointers                 526.0  0.007226  0.123102 -0.400 -0.03075  0.001   
Greater Than 15 Ft         531.0  0.003154  0.112102 -0.441 -0.02600  0.003   
Less Than 10 Ft            525.0  0.021015  0.125468 -0.754 -0.03700  0.018   
Less Than 6 Ft             523.0  0.030228  0.138380 -0.660 -0.03850  0.022   
Overall                    534.0  0.007170  0.088566 -0.531 -0.02275  0.002   

                                    FREQ                                     \
                       75%    max  count      mean       std    min     25%   
DEFENSE_CATEGORY                                                              
2 Pointers          0.0430  0.552  529.0  0.653221  0.113874  0.294  0.5800   
3 Pointers          0.0350  0.725  526.0  0.358266  0.123051  0.050  0.2860   
Greater Than 15 Ft  0.0335  0.675  531.0  0.506194  0.130201  0.100  0.4305   
Less Than 10 Ft     0.0710  0.565  525.0  0.428076  0.126936  0.091  0.3330   
Less Than 6 Ft      0.0830  0.558  523.0  0.327956  0.115921  0.091  0.2500   
Overall             0.0320  0.640  534.0  1.000000  0.000000  1.000  1.0000   

                                          
                       50%      75%  max  
DEFENSE_CATEGORY                          
2 Pointers          0.6340  0.72000  1.0  
3 Pointers          0.3685  0.42275  1.0  
Greater Than 15 Ft  0.5200  0.58400  1.0  
Less Than 10 Ft     0.4000  0.50000  1.0  
Less Than 6 Ft      0.3070  0.40150  1.0  
Overall             1.0000  1.00000  1.0

In [20]:
def defensiveUnstack(season,season_type,measures):
    endpt = 'playerdashptshotdefend'
    params = {'DateFrom':'','DateTo':'','GameSegment':'',
          'LastNGames':'0','LeagueID':'00','Location':'',
          'Month':'0','OpponentTeamID':'0','Outcome':'',
          'Period':'0','PlayerID':'0','Season':year,
          'SeasonSemgent':'','SeasonType':'Regular Season',
          'TeamID':'0','VsConference':'','VsDivision':'',
          'PerMode':'Totals','SeasonSegment':''
         }
    params.update({'Season':year,
                   'SeasonType':season_type                  
                  })
    defensive = get_nba_data(endpt=endpt,params=params)
    defensive['PLAYER_ID']=defensive.CLOSE_DEF_PERSON_ID
    defensive['SEASON'] = int(season[:4])+1
    defensive['SEASON_TYPE'] = season_type
    defensive = defensive.set_index('CLOSE_DEF_PERSON_ID')

    categories = defensive.DEFENSE_CATEGORY.unique()[1:]
    
    defensive_seperate = [defensive[defensive.DEFENSE_CATEGORY==cat][measures] for cat in categories]
    suffix_list = ['_3Pt',"_2Pt","_Lt6","_Lt10","_Gt15"]
    col_pair = defensive_seperate[0].columns
    cols = ["DEF_"+col_pair + suffix for suffix in suffix_list]
    
    for i,pair in enumerate(cols):
        defensive_seperate[i].columns=pair

    return(pd.concat(defensive_seperate,axis=1))

defensiveUnstack('2016-17','Playoffs',['PCT_PLUSMINUS','FREQ']).head()

,DEF_PCT_PLUSMINUS_3Pt,DEF_FREQ_3Pt,DEF_PCT_PLUSMINUS_2Pt,DEF_FREQ_2Pt,DEF_PCT_PLUSMINUS_Lt6,DEF_FREQ_Lt6,DEF_PCT_PLUSMINUS_Lt10,DEF_FREQ_Lt10,DEF_PCT_PLUSMINUS_Gt15,DEF_FREQ_Gt15
CLOSE_DEF_PERSON_ID,,,,,,,,,,
201142,-0.019,0.418,-0.040,0.582,-0.069,0.352,-0.046,0.449,-0.032,0.504
201580,-0.068,0.301,-0.045,0.699,-0.209,0.280,-0.082,0.409,-0.001,0.516
1628395,-0.007,0.375,-0.147,0.625,-0.183,0.398,-0.151,0.489,-0.013,0.443
2738,0.036,0.484,-0.054,0.516,0.042,0.263,0.053,0.347,-0.007,0.558
203110,-0.083,0.286,-0.070,0.714,-0.105,0.427,-0.075,0.527,-0.098,0.395


In [21]:
defensive = defensiveUnstack('2017-18','Playoffs',['PCT_PLUSMINUS','FREQ']).head()
defensive_full= pd.DataFrame(columns=defensive.columns)

for i in range(13,18):
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({'Season':year})
    reg_season = defensiveUnstack(year,'Regular Season',['PCT_PLUSMINUS','FREQ'])
    reg_season['SEASON_TYPE'] = "Regular Season"
    playoff = defensiveUnstack(year,'Playoffs',['PCT_PLUSMINUS','FREQ'])
    playoff['SEASON_TYPE'] = "Playoffs"
    
    merged = pd.concat([reg_season,playoff])
    merged['PLAYER_ID']=merged.index
    merged['SEASON'] = 2001+i
    
    defensive_full = pd.concat([defensive_full,merged])


defensive_full['SEASON']=pd.Series(defensive_full.SEASON,dtype='int64')

defensive_full['PLAYER_ID']=pd.Series(defensive_full.PLAYER_ID,dtype='int64')
defensive_full['PLAYER_ID']=pd.Series(defensive_full.PLAYER_ID,dtype='str')

defensive_full = defensive_full.set_index(['PLAYER_ID','SEASON','SEASON_TYPE'])

defensive_full.sort_index().head(15)

DEF_FREQ_2Pt  DEF_FREQ_3Pt  DEF_FREQ_Gt15  \
PLAYER_ID SEASON SEASON_TYPE                                                 
101106    2014   Regular Season         0.936         0.064          0.267   
          2015   Playoffs               0.945         0.055          0.235   
                 Regular Season         0.935         0.065          0.258   
          2016   Playoffs               0.869         0.131          0.193   
                 Regular Season         0.918         0.082          0.265   
          2017   Regular Season         0.916         0.084          0.248   
          2018   Regular Season         0.823         0.177          0.278   
101107    2014   Regular Season         0.737         0.263          0.435   
          2015   Regular Season         0.755         0.245          0.460   
          2016   Playoffs               0.633         0.367          0.517   
                 Regular Season         0.707         0.293          0.472   
          2017   Regular Season         0.638         0.362          0.526   
          2018   Regular Season         0.669         0.331          0.501   
101108    2014   Playoffs               0.617         0.383          0.555   
                 Regular Season         0.648         0.352          0.558   

                                 DEF_FREQ_Lt10  DEF_FREQ_Lt6  \
PLAYER_ID SEASON SEASON_TYPE                                   
101106    2014   Regular Season          0.628         0.483   
          2015   Playoffs                0.680         0.510   
                 Regular Season          0.672         0.489   
          2016   Playoffs                0.731         0.586   
                 Regular Season          0.635         0.491   
          2017   Regular Season          0.650         0.500   
          2018   Regular Season          0.620         0.430   
101107    2014   Regular Season          0.478         0.342   
          2015   Regular Season          0.457         0.332   
          2016   Playoffs                0.400         0.250   
                 Regular Season          0.463         0.379   
          2017   Regular Season          0.403         0.314   
          2018   Regular Season          0.423         0.336   
101108    2014   Playoffs                0.328         0.258   
                 Regular Season          0.338         0.249   

                                 DEF_PCT_PLUSMINUS_2Pt  DEF_PCT_PLUSMINUS_3Pt  \
PLAYER_ID SEASON SEASON_TYPE                                                    
101106    2014   Regular Season                 -0.052                 -0.056   
          2015   Playoffs                       -0.049                 -0.067   
                 Regular Season                 -0.075                 -0.008   
          2016   Playoffs                       -0.112                 -0.117   
                 Regular Season                 -0.051                 -0.005   
          2017   Regular Season                 -0.052                  0.187   
          2018   Regular Season                 -0.112                  0.062   
101107    2014   Regular Season                  0.010                 -0.005   
          2015   Regular Season                 -0.019                 -0.046   
          2016   Playoffs                        0.059                  0.007   
                 Regular Season                  0.001                 -0.006   
          2017   Regular Season                 -0.021                  0.005   
          2018   Regular Season                  0.031                 -0.046   
101108    2014   Playoffs                        0.035                 -0.001   
                 Regular Season                  0.015                 -0.010   

                                 DEF_PCT_PLUSMINUS_Gt15  \
PLAYER_ID SEASON SEASON_TYPE                              
101106    2014   Regular Season                  -0.004   
          2015   Playoffs                         0.055   
   

In [167]:
x = pd.merge(advanced_full,per_100p_full,left_index=True,right_index=True)
y = pd.merge(x, scoring_full,left_index=True,right_index=True)
all_full = pd.merge(y,defensive_full,left_index=True,right_index=True)
len(all_full.columns)
all_full['SEASON_MIN']=all_full.GP*all_full.MIN

In [168]:
pickle.dump(all_full, open('NBA_2014_18_Adv_100_Score_Def.pkl', 'wb'))

In [114]:

idx = pd.IndexSlice
all_full_reg_season = all_full.loc[idx[:,:, 'Regular Season'],:]
qualified = all_full_reg_season.query("SEASON_MIN>800")
qualified

PLAYER_NAME   AGE  AST_PCT  AST_RATIO  \
PLAYER_ID SEASON SEASON_TYPE                                                  
101106    2014   Regular Season      Andrew Bogut  29.0    0.087       18.3   
          2015   Regular Season      Andrew Bogut  30.0    0.149       27.3   
          2016   Regular Season      Andrew Bogut  31.0    0.145       29.7   
101107    2014   Regular Season   Marvin Williams  28.0    0.075       11.3   
          2015   Regular Season   Marvin Williams  29.0    0.078       14.4   
          2016   Regular Season   Marvin Williams  30.0    0.074       11.2   
          2017   Regular Season   Marvin Williams  31.0    0.072       11.3   
          2018   Regular Season   Marvin Williams  31.0    0.069       12.3   
101108    2014   Regular Season        Chris Paul  29.0    0.459       36.2   
          2015   Regular Season        Chris Paul  30.0    0.441       35.8   
          2016   Regular Season        Chris Paul  31.0    0.493       33.7   
          2017   Regular Season        Chris Paul  32.0    0.444       35.0   
          2018   Regular Season        Chris Paul  33.0    0.389       30.9   
101109    2014   Regular Season    Raymond Felton  30.0    0.275       31.6   
          2016   Regular Season    Raymond Felton  32.0    0.205       24.4   
          2017   Regular Season    Raymond Felton  33.0    0.172       23.4   
          2018   Regular Season    Raymond Felton  33.0    0.255       23.9   
101110    2014   Regular Season   Martell Webster  27.0    0.068       12.2   
101112    2014   Regular Season     Channing Frye  31.0    0.061        9.7   
          2015   Regular Season     Channing Frye  32.0    0.074       13.9   
          2016   Regular Season     Channing Frye  33.0    0.086       14.9   
          2017   Regular Season     Channing Frye  34.0    0.050        6.9   
101114    2014   Regular Season    Deron Williams  30.0    0.312       29.1   
          2015   Regular Season    Deron Williams  31.0    0.326       30.3   
          2016   Regular Season    Deron Williams  32.0    0.295       27.1   
          2017   Regular Season    Deron Williams  33.0    0.350       31.4   
101122    2014   Regular Season     Danny Granger  31.0    0.080        9.7   
101123    2014   Regular Season      Gerald Green  28.0    0.085        8.9   
          2015   Regular Season      Gerald Green  29.0    0.110        8.9   
          2016   Regular Season      Gerald Green  30.0    0.057        7.7   
...                                           ...   ...      ...        ...   
2747      2017   Regular Season          JR Smith  31.0    0.071       13.8   
          2018   Regular Season          JR Smith  32.0    0.087       16.8   
2748      2014   Regular Season     Dorell Wright  28.0    0.096       14.9   
2749      2014   Regular Season     Jameer Nelson  32.0    0.341       33.1   
          2015   Regular Season     Jameer Nelson  33.0    0.278       28.7   
          2016   Regular Season     Jameer Nelson  34.0    0.287       32.8   
          2017   Regular Season     Jameer Nelson  35.0    0.258       33.6   
          2018   Regular Season     Jameer Nelson  36.0    0.255       35.9   
2754      2014   Regular Season        Tony Allen  32.0    0.115       14.5   
          2015   Regular Season        Tony Allen  33.0    0.081       12.6   
          2016   Regular Season        Tony Allen  34.0    0.068       10.3   
          2017   Regular Season        Tony Allen  35.0    0.083       11.5   
2755      2014   Regular Season      Kevin Martin  31.0    0.090        8.6   
          2015   Regular Season      Kevin Martin  32.0    0.123       10.2   
          2016   Regular Season      Kevin Martin  33.0    0.089        9.6   
2756      2016   Regular Season     Sasha Vujacic  32.0    0.146       19.9   
2757      2015   Regular Season        Beno Udrih  32.0    0.236       25.6   
2760      2014   Regular Season  Anderson Varejao  31.0    0.122       19.2   
2772     

In [143]:
all_full.columns
advancedVars = ['AST_PCT','AST_RATIO','AST_TO','DEF_RATING','DREB_PCT',
                'EFG_PCT','OFF_RATING','OREB_PCT','PACE','REB_PCT',
                'TM_TOV_PCT','TS_PCT','USG_PCT']

per100pVars_shots = ['FG3A','FG3M','FG3_PCT','FGA','FGM','FG_PCT',
                     'FTA','FTM','FT_PCT'
                    ]
per100pVars_basic = ['AST','BLK','DREB','OREB','STL','PTS','PF','TOV']
per100pVars_other= ['BLKA','PFD','PLUS_MINUS']

scoringVars = list(all_full.columns[40:55])
defensiveVars = list(all_full.columns[56:66])
X = np.array(qualified[advancedVars+per100pVars_shots+
                       scoringVars+defensiveVars])


In [149]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [156]:
X_scaled = scale(X)
pca = PCA(n_components=10)
pca.fit(X_scaled)
print(pca.components_)

[[ -1.27815240e-01  -1.25528058e-01  -1.58576512e-01  -2.90150558e-02
    2.03363073e-01   8.36994228e-02  -2.36005997e-02   2.34722079e-01
   -2.82944639e-02   2.27937930e-01   8.90216725e-02   6.54144071e-02
   -4.84737653e-02  -2.06158541e-01  -1.98672126e-01  -1.71595770e-01
   -6.63932561e-02   1.79135776e-02   1.93230892e-01   5.03556145e-02
    6.29363781e-03  -1.43318970e-01   1.59470000e-01  -1.22902005e-01
    9.60381032e-02   1.99594456e-01  -1.99595010e-01   1.99591909e-01
   -3.10122378e-02  -1.95285008e-01  -1.13046606e-01   4.90380112e-02
   -6.69341149e-02   2.07465921e-01  -1.59470318e-01  -1.20636082e-01
   -9.60410852e-02  -2.18446037e-01  -2.26602664e-01   2.28030055e-01
    2.23555768e-01  -1.12270306e-01   1.96074009e-02   3.23371735e-02
   -1.62294982e-01  -1.76551535e-01  -3.35093928e-02]
 [  2.43730062e-01   1.12048150e-01   7.40369593e-02   3.36573063e-02
    3.64546192e-02  -8.20156435e-02   2.41951485e-02   2.67165900e-02
   -3.71450421e-03   3.47037200e-02 

/Users/davidhoskins/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [162]:
np.cumsum(pca.explained_variance_ratio_)

array([ 0.32380658,  0.48190144,  0.57513212,  0.63272856,  0.68156526,
        0.72615479,  0.76474253,  0.79487646,  0.8223687 ,  0.84504501])